In [1]:
from nanomesh.generator import Generator
import SimpleITK as sitk
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image
import numpy as np
import math
import pygalmesh

In [2]:

class Pore3D(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.gen = Generator(680, math.sqrt(2)*680, 0.24*680)

        # Possible rotation/transformation of the coordinate system
        theta = math.pi * 1/180
        c = math.cos(theta)
        s = math.sin(theta)
        trans = np.array([
            [ c, 0, s],
            [ 0, 1, 0],
            [-s, 0, c]
        ])
        
        self.size = [100]*3
        self.res = [10]*3
        self.data = self.gen.generate_vect(self.size, 
                                           self.res, 
                                           transform=trans, 
                                           bin_val=[0.,1.])
        self.data = self.data[:,:68,:]
                
        self.x = np.linspace(0,1.,self.data.shape[0])
        self.y = np.linspace(0,0.68,self.data.shape[1])
        self.z = np.linspace(0,1.,self.data.shape[2])
        
    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.y))
        iz = np.argmin(np.abs(x[2]-self.z))
        if self.data[ix,iy,iz] == 1:
            out = -1.
        else:
            out = 1.
        return out
    

In [3]:
pore = Pore3D()
im = sitk.GetImageFromArray(pore.data.astype('uint8'))
show_slice(im, dim='z',scale=2)

interactive(children=(IntSlider(value=49, description='z', max=99), Output()), _dom_classes=('widget-interact'…

In [4]:
class FullCube(pygalmesh.DomainBase):
    
    def __init__(self):
        super().__init__()

    def eval(self,x):
        return -1

In [6]:
mesh = pygalmesh.generate_periodic_mesh_multiple_domains(
    [Pore3D(),FullCube()],
    ["--","+-"],
    [0,0,0,1.,0.68,1.],
    max_cell_circumradius=0.05,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.05,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=1,
    # odt=True,
    # lloyd=True,
    verbose=False,
    )

In [7]:
import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [8]:
from nanomesh.periodic_utils import insert_periodic_info
mesh = insert_periodic_info(mesh, [0,0,0,1.,0.68,1.])

In [9]:
mesh.write('nanopt_full_period_2domain.gmsh', file_format="gmsh22", binary=False)